In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156230 sha256=cca1ce452de7c916a4ddae886d6fa83eca2fce31bf90594d9732af408f43aa1c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
!gdown 1C0Qn6FUwNQtq5MowDF3Eheg5ilE4pPIB
!gdown 1FpvYObY52Wsl-3kYVRd_lNHWSAMVLY1U

Downloading...
From: https://drive.google.com/uc?id=1C0Qn6FUwNQtq5MowDF3Eheg5ilE4pPIB
To: /content/movies.csv
100% 494k/494k [00:00<00:00, 126MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FpvYObY52Wsl-3kYVRd_lNHWSAMVLY1U
To: /content/ratings.csv
100% 2.48M/2.48M [00:00<00:00, 157MB/s]


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
df.shape

(100836, 4)

In [ ]:
df.userId.nunique() # 610 unique user ids

610

In [ ]:
df.movieId.nunique() # 9724 unique movies

9724

In [ ]:
# suprive is used to develop recommendation system

In [ ]:
# we need collabortive Filtering Algorithms

In [ ]:
X = df.iloc[:, :-1]
X.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
X.rating.min(), X.rating.max()

(0.5, 5.0)

In [ ]:
from surprise import accuracy, Dataset, SVD, Reader
from surprise.model_selection import train_test_split

In [ ]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(X, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.25)

In [ ]:
model = SVD()
model.fit(trainset)

In [ ]:
predictions = model.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.8818


0.8818438024292775

In [ ]:
# model.predict(uid=1, iid = 15)

In [ ]:
X.loc[X.userId == 120]

,userId,movieId,rating
18751,120,5,4.0
18752,120,12,3.0
18753,120,32,3.0
18754,120,52,3.0
18755,120,141,3.0
18756,120,260,5.0
18757,120,608,5.0
18758,120,631,3.0
18759,120,648,3.0
18760,120,653,3.0


In [ ]:
model.predict(uid=120, iid=2) # how much rating will user id 120 give to movie id 2

Prediction(uid=120, iid=2, r_ui=None, est=3.5162597060591962, details={'was_impossible': False})

In [ ]:
movie = pd.read_csv('/content/movies.csv')
movie.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [ ]:
all_data = pd.merge(df, movie, on='movieId', how = 'inner')

In [ ]:
all_data

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [ ]:
pd.__version__

'1.5.3'

# Top n recommender

In [ ]:
inference_data = all_data[['userId', 'movieId', 'rating', 'title']]

In [ ]:
model.predict(uid=120, iid=2).est

3.5162597060591962

In [ ]:
def recommend_top_n(model, uid, inference_data = inference_data, n = 10):
    rate = list()
    movieids = inference_data.movieId.unique()
    unseen_movieids = list(set(movieids) - set(inference_data.loc[inference_data.userId == uid].movieId))
    for item in unseen_movieids:
        rate.append(model.predict(uid=uid, iid=item).est)
    recommendations = pd.DataFrame({'movieId': unseen_movieids, 'estimated_ratings': rate})
    return recommendations

In [ ]:
recommendations = recommend_top_n(model, uid=120)

In [ ]:
top10 = recommendations.sort_values(by=['estimated_ratings'], ascending = False).head(10)

In [ ]:
movieid_title = movie[['movieId', 'title']]

In [ ]:
pd.merge(top10, movieid_title, on = 'movieId', how = 'inner')

,movieId,estimated_ratings,title
0,750,4.740376,Dr. Strangelove or: How I Learned to Stop Worr...
1,912,4.476114,Casablanca (1942)
2,3108,4.450114,"Fisher King, The (1991)"
3,1206,4.446979,"Clockwork Orange, A (1971)"
4,318,4.417953,"Shawshank Redemption, The (1994)"
5,3275,4.407887,"Boondock Saints, The (2000)"
6,1204,4.406163,Lawrence of Arabia (1962)
7,6711,4.386677,Lost in Translation (2003)
8,5618,4.385618,Spirited Away (Sen to Chihiro no kamikakushi) ...
9,1252,4.385278,Chinatown (1974)
